## VGG16 모델 활용 이미지 식별(CIFAR10 데이터셋)

In [5]:
from keras.utils import set_random_seed
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from keras.datasets.cifar10 import load_data

from keras.applications import VGG16

import numpy as np
import matplotlib.pyplot as plt

In [3]:
set_random_seed(11)

In [4]:
# 학습용 : 테스트용 = 8 : 2
(x_train, y_train), (x_test, y_test) = load_data()

In [8]:
# 스케일링
x_train=x_train/255.0
x_test=x_test/255.0

In [9]:
x_train.shape

(50000, 32, 32, 3)

In [28]:
# 원핫인코딩으로 변환
from sklearn.preprocessing import OneHotEncoder

# y_train
encoder=OneHotEncoder()
encoder.fit(y_train.reshape(-1,1))
y_train=encoder.transform(y_train.reshape(-1,1))
y_train=y_train.toarray()

# y_test
encoder.fit(y_test.reshape(-1,1))
y_test=encoder.transform(y_test.reshape(-1,1))
y_test=y_test.toarray()

In [20]:
# VGG16 모델

# 모델 생성(이미지넷으로 학습된 모델, 입력 형태 변경)
model_vgg=VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

# 이미지넷으로 학습된 값 그대로 사용
model_vgg.trainable = False 

model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [48]:
# 모델 생성
model=Sequential()

# vgg16모델 
model.add(model_vgg)

# 형태 변환
model.add(Flatten())

model.add(Dense(2000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))

# 출력층
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_8 (Flatten)         (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 2000)              1026000   
                                                                 
 dense_22 (Dense)            (None, 1000)              2001000   
                                                                 
 dropout_2 (Dropout)         (None, 1000)              0         
                                                                 
 dense_23 (Dense)            (None, 500)               500500    
                                                                 
 dense_24 (Dense)            (None, 100)              

In [49]:
# 모델 생성
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [55]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 검증 데이터에 대한 loss값 2번 연속 개선되지 않으면 학습 중지
stopCB=EarlyStopping(monitor='val_loss', patience=3)

In [58]:
# 모델 학습
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test), callbacks=[stopCB])

Epoch 1/10
1563/1563 [==============================] - 457s 292ms/step - loss: 0.6881 - accuracy: 0.7566 - val_loss: 1.2495 - val_accuracy: 0.6176
Epoch 2/10
1563/1563 [==============================] - 420s 269ms/step - loss: 0.6574 - accuracy: 0.7660 - val_loss: 1.3326 - val_accuracy: 0.6152
Epoch 3/10
1563/1563 [==============================] - 478s 306ms/step - loss: 0.6351 - accuracy: 0.7749 - val_loss: 1.3391 - val_accuracy: 0.6195
Epoch 4/10
1563/1563 [==============================] - 476s 305ms/step - loss: 0.6075 - accuracy: 0.7848 - val_loss: 1.3725 - val_accuracy: 0.6111


In [59]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 51s 163ms/step - loss: 1.3725 - accuracy: 0.6111


[1.3724864721298218, 0.6111000180244446]

In [46]:
# # 모델 저장
# from keras.models import save_model
# model061='cifar10/Model061.h5'
# save_model(model, model061)

In [47]:
# # 모델 복원
# from keras.models import load_model
# reModel=load_model(model061)

In [ ]:
# 모델 학습
# reModel.fit(x_train, y_train, epochs=10)

In [60]:
# 모델 생성
model2=Sequential()

# vgg16모델 
model2.add(model_vgg)

# 형태 변환
model2.add(Flatten())

# 출력층
model2.add(Dense(10, activation='softmax'))

model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_9 (Flatten)         (None, 512)               0         
                                                                 
 dense_26 (Dense)            (None, 10)                5130      
                                                                 
Total params: 14,719,818
Trainable params: 5,130
Non-trainable params: 14,714,688
_________________________________________________________________


In [63]:
# 모델 생성
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [64]:
# 검증 데이터에 대한 loss값 2번 연속 개선되지 않으면 학습 중지
stopCB2=EarlyStopping(monitor='val_loss', patience=5)

In [65]:
# 모델 학습
model2.fit(x_train, y_train, epochs=250, batch_size=256, validation_data=(x_test, y_test), callbacks=[stopCB2])

Epoch 1/250
 76/196 [==========>...................] - ETA: 2:34 - loss: 1.2031 - accuracy: 0.5875